<h1 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4); 
           color: white; 
           padding: 20px; 
           border-radius: 10px; 
           text-align: center; 
           font-family: Arial, sans-serif; 
           text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
  Langchain with Amazon Bedrock
</h1>

## Introduction to LangChain 

LangChain is a framework for developing applications powered by language models. It provides a suite of tools and components to streamline the creation of complex applications that leverage the capabilities of large language models (LLMs). LangChain simplifies the integration of LLMs into various applications, enabling developers to build sophisticated language-based solutions with ease.

Amazon Bedrock is a fully managed service that makes it easy to build, train, and deploy machine learning models at scale. By integrating LangChain with Amazon Bedrock, developers can harness the power of both platforms to create robust and scalable language model applications. This combination allows for seamless deployment and management of LLMs, providing a powerful solution for developing advanced language-based applications.


<div style="background-color:#f0f8ff; padding: 15px; border-radius: 10px;">
  <p>An alternative is to use the SDK of the LLM provider you started with, like <code>boto3</code> for AWS. However, learning LangChain offers clear benefits:</p>
  
  <ol>
    <li><b>Ease of Use:</b> Simplifies working with LLMs by abstracting API complexities, reducing code.</li>
    <li><b>Flexibility:</b> Supports multiple LLM providers, making it easy to switch services.</li>
    <li><b>Integration:</b> Works well with popular libraries like PyTorch and TensorFlow.</li>
    <li><b>Community:</b> Large, active community with frequent updates and support.</li>
  </ol>

  <h4>Prebuilt Patterns</h4>
  <p>LangChain provides common LLM patterns (e.g., chain-of-thought) to help you quickly get started. Use these to see if they meet your needs before diving into more complex implementations.</p>

  <h4>Interchangeable Components</h4>
  <p>LangChain components (e.g., LLMs, output parsers) are easily swapped, future-proofing your application as models and needs evolve.</p>
</div>


## Installation

In [1]:
# %pip install langchain-community
# %pip install langchain-aws 

In [2]:
import boto3
from tabulate import tabulate

# Create a client for Bedrock
client = boto3.client('bedrock', region_name='us-east-1')  # Replace with your region

# Call an API to list available models
response = client.list_foundation_models()

# Print the available models with modalities as strings
table_data = [[
    model['modelName'],
    model['modelId'],
    ', '.join(model['inputModalities']),
    ', '.join(model['outputModalities'])
] for model in response['modelSummaries']]

print(tabulate(table_data, headers=['Model Name', 'Model ID', 'Input Modalities', 'Output Modalities'], tablefmt='grid', colalign=('left', 'left')))

+--------------------------------+------------------------------------------------+--------------------+---------------------+
| Model Name                     | Model ID                                       | Input Modalities   | Output Modalities   |
+================================+================================================+====================+=====================+
| Titan Text Large               | amazon.titan-tg1-large                         | TEXT               | TEXT                |
+--------------------------------+------------------------------------------------+--------------------+---------------------+
| Titan Image Generator G1       | amazon.titan-image-generator-v1:0              | TEXT, IMAGE        | IMAGE               |
+--------------------------------+------------------------------------------------+--------------------+---------------------+
| Titan Image Generator G1       | amazon.titan-image-generator-v1                | TEXT, IMAGE        | IMAGE 

## Chat models vs LLM in Langchain

<div style="background-color:#f0f8ff; padding: 15px; border-radius: 10px;">
  <p><b>LangChain</b> offers two simple interfaces to interact with any LLM API provider:</p>
  
  <ul>
    <li><b>LLMs</b></li>
    <li><b>Chat models</b></li>
  </ul>
</div>


In [3]:
# Using BedrockLLM (Not Recommended Now)
from langchain_aws import BedrockLLM

llm = BedrockLLM(model_id="amazon.titan-text-premier-v1:0")

prompt = "What are the best books on Deep Learning? May be top 5"
completion = llm.invoke(prompt)

print(completion)

Here are five highly recommended books on Deep Learning:

1. "Deep Learning" by Ian Goodfellow, Yoshua Bengio, and Aaron Courville: This book is a comprehensive and in-depth guide to deep learning, covering topics such as neural networks, convolutional neural networks, recurrent neural networks, and deep reinforcement learning. It is widely regarded as one of the best resources for understanding the theoretical foundations of deep learning.

2. "Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow" by Aurélien Géron: This book provides a practical introduction to machine learning and deep learning using Python and popular libraries such as Scikit-Learn, Keras, and TensorFlow. It covers a wide range of topics, from basic machine learning concepts to advanced deep learning techniques, and includes many examples and exercises to help you apply what you've learned.

3. "Neural Networks and Deep Learning" by Michael Nielsen: This book is an online tutorial that provides a gent

In [4]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0")

prompt = "What are the best books on Linear Algebra? May be top 2"
completion = llm.invoke(prompt)
print(completion.content)

When it comes to linear algebra books, two titles that are highly regarded and widely recommended are:

1. "Linear Algebra and Its Applications" by Gilbert Strang (5th Edition)
This book is often considered one of the best introductory texts on linear algebra. It is known for its clear explanations, well-chosen examples, and its emphasis on the geometric interpretation of linear algebra concepts. Strang's book covers a wide range of topics, including vector spaces, matrices, determinants, eigenvalues and eigenvectors, and applications to various fields.

2. "Linear Algebra Done Right" by Sheldon Axler
This book takes a more theoretical and rigorous approach to linear algebra. It is often recommended for advanced undergraduate or graduate-level students who want a deeper understanding of the subject. Axler's book focuses on developing a solid foundation in linear algebra, with a strong emphasis on proofs and abstract concepts. It covers topics such as vector spaces, linear transformatio

In [5]:
llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", model_kwargs={"temperature": .7, "max_tokens": 100})

prompt = "What are the best books on Linear Algebra? May be top 2"
completion = llm.invoke(prompt)
print(completion.content)

Here are two of the best and most widely recommended books on Linear Algebra:

1. "Linear Algebra and Its Applications" by Gilbert Strang (5th Edition, 2016)
This book is considered one of the most comprehensive and accessible introductions to linear algebra. It covers a wide range of topics, including matrices, vector spaces, linear transformations, eigenvalues and eigenvectors, and numerical methods. The book is known for its clear explanations, numerous


In [6]:
completion.response_metadata

{'usage': {'prompt_tokens': 22, 'completion_tokens': 100, 'total_tokens': 122},
 'stop_reason': 'max_tokens',
 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}

## Prompt Template

#### General purpose `PromptTemplate`

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Give me a one line definition of {word} with {num} of examples")
prompt = prompt_template.format(word="flabbergasted", num=2)

print(response)

{'ResponseMetadata': {'RequestId': 'da1df7ac-ca40-442e-b194-b605fd74c7a1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 11 Sep 2024 19:36:37 GMT', 'content-type': 'application/json', 'content-length': '30408', 'connection': 'keep-alive', 'x-amzn-requestid': 'da1df7ac-ca40-442e-b194-b605fd74c7a1'}, 'RetryAttempts': 0}, 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large', 'modelId': 'amazon.titan-tg1-large', 'modelName': 'Titan Text Large', 'providerName': 'Amazon', 'inputModalities': ['TEXT'], 'outputModalities': ['TEXT'], 'responseStreamingSupported': True, 'customizationsSupported': [], 'inferenceTypesSupported': ['ON_DEMAND'], 'modelLifecycle': {'status': 'ACTIVE'}}, {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0', 'modelId': 'amazon.titan-image-generator-v1:0', 'modelName': 'Titan Image Generator G1', 'providerName': 'Amazon', 'inputModalities': ['TEXT', 'IMAGE'], 'outputModalities'

In [8]:
llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0")
completion = llm.invoke(prompt)

print(completion.content)

Flabbergasted means to be utterly astonished or astounded, as in "She was flabbergasted when she won the lottery jackpot" or "The magician's trick left the audience flabbergasted."


<div style="background-color:#f0f8ff; padding: 15px; border-radius: 10px; border-left: 6px solid #4682B4;">
  <p>Alternatively, the <b>Chat Model interface</b> enables back-and-forth conversations between the user and the model. This interface is separate because popular LLM providers, like OpenAI, differentiate messages into <b>user</b>, <b>assistant</b>, and <b>system roles</b>, which define the type of content each message contains:</p>
  
  <ul>
    <li><b>System role:</b> Specifies instructions the model should use to answer a user question.</li>
    <li><b>User role:</b> The individual asking questions and generating queries sent to the model.</li>
    <li><b>Assistant role:</b> The model’s responses to the user’s query.</li>
  </ul>
</div>


In [9]:
from langchain_core.messages import HumanMessage
from langchain_aws import ChatBedrock

llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0")

prompt = [HumanMessage(content="What is the capital of India?")]
completion = llm.invoke(prompt)

print(completion.content)


The capital of India is New Delhi.


<div style="background-color:#f0f8ff; padding: 15px; border-radius: 10px; border-left: 6px solid #4682B4;">
  <p>Instead of a single prompt string, <b>chat models</b> use different types of chat message interfaces, each associated with a specific role:</p>
  
  <ul>
    <li><b>HumanMessage:</b> Sent from the human's perspective, with the user role.</li>
    <li><b>AIMessage:</b> Sent from the AI's perspective, with the assistant role.</li>
    <li><b>SystemMessage:</b> Provides instructions the AI should follow, with the system role.</li>
    <li><b>ChatMessage:</b> Allows arbitrary role setting.</li>
  </ul>
</div>


In [10]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_aws import ChatBedrock

llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", model_kwargs={"temperature": .7, "max_tokens": 100})

prompt = [
            SystemMessage(content="You are a helpful assistant that answers questions with a joke at the end."),
            HumanMessage(content="Who was the president of the United States in 2010?")
         ]
completion = llm.invoke(prompt)
completion

AIMessage(content='Barack Obama was the president of the United States in 2010. He served two terms from 2009 to 2017. Why was the baseball player so cool? Because he had a great pitch!', additional_kwargs={'usage': {'prompt_tokens': 35, 'completion_tokens': 46, 'total_tokens': 81}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, response_metadata={'usage': {'prompt_tokens': 35, 'completion_tokens': 46, 'total_tokens': 81}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, id='run-7cf0903d-3761-4cc3-936a-3672489bc42d-0', usage_metadata={'input_tokens': 35, 'output_tokens': 46, 'total_tokens': 81})

As you can see, the model obeyed the instruction provided in the SystemMessage even though it wasn’t present in the user’s question. This enables you to pre-configure your AI application to respond in a relatively predictable manner based on the user’s input.


#### Using `ChatPromptTemplate`

In [11]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that gives a one-line definition of the word entered by user"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(user_input="Sesquipedalian")
messages

[SystemMessage(content='You are a helpful assistant that gives a one-line definition of the word entered by user'),
 HumanMessage(content='Sesquipedalian')]

In [12]:
llm.invoke(messages)

AIMessage(content='Sesquipedalian means using long words unnecessarily.', additional_kwargs={'usage': {'prompt_tokens': 31, 'completion_tokens': 16, 'total_tokens': 47}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, response_metadata={'usage': {'prompt_tokens': 31, 'completion_tokens': 16, 'total_tokens': 47}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, id='run-ab557c17-a704-4937-aadd-1a66cfbdd83b-0', usage_metadata={'input_tokens': 31, 'output_tokens': 16, 'total_tokens': 47})

In [13]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that gives a one-line answer to user query"),
        ("human", "Who created theory of relativity?"),
        ("ai", "Albert Einstein developed the theory of relativity."),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(user_input="When was it created?")
messages

[SystemMessage(content='You are a helpful assistant that gives a one-line answer to user query'),
 HumanMessage(content='Who created theory of relativity?'),
 AIMessage(content='Albert Einstein developed the theory of relativity.'),
 HumanMessage(content='When was it created?')]

In [14]:
llm.invoke(messages)

AIMessage(content='The theory of relativity was developed by Einstein in the early 20th century.', additional_kwargs={'usage': {'prompt_tokens': 49, 'completion_tokens': 20, 'total_tokens': 69}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, response_metadata={'usage': {'prompt_tokens': 49, 'completion_tokens': 20, 'total_tokens': 69}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, id='run-2dd3225a-6463-42cb-a672-af6e1e19fe44-0', usage_metadata={'input_tokens': 49, 'output_tokens': 20, 'total_tokens': 69})

## Getting Specific Formats out of LLMs

Plain text outputs are useful, but there may be use cases where you need the LLM to generate a structured output, that is, output in a machine-readable format, such as JSON, XML or CSV, or even in a programming language such as Python or JavaScript. This is very useful when you intend to hand that output off to some other piece of code, making an LLM play a part in your larger application.

#### JSON Output

In [15]:
from langchain_aws import ChatBedrock
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import HumanMessage
from langchain_aws import ChatBedrock

class Movie(BaseModel):
    title: str = Field(description="The title of the movie")
    director: str = Field(description="The director of the movie")
    release_year: int = Field(description="The year the movie was released")
    description: str = Field(description="A brief description of the movie")

llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", model_kwargs={"temperature": .1})
structured_llm = llm.with_structured_output(Movie)

prompt = [HumanMessage(content="What is a highly acclaimed science fiction movie?")]
completion = structured_llm.invoke(prompt)

from pprint import pprint
pprint(completion)

Movie(title='Interstellar', director='Christopher Nolan', release_year=2014, description='Interstellar is a 2014 science fiction film directed by Christopher Nolan. It depicts a crew of astronauts who travel through a newly discovered wormhole in search of habitable planets for humanity to colonize, as life on Earth is becoming unsustainable. The film stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, and Michael Caine, and received widespread critical acclaim for its visuals, musical score, and exploration of scientific theories like relativity and wormholes.')


In [16]:
llm.invoke(prompt).content


'One highly acclaimed science fiction movie is "Interstellar" directed by Christopher Nolan.\n\nSome key details about Interstellar:\n\n- Released in 2014\n- Stars Matthew McConaughey, Anne Hathaway, Jessica Chastain\n- About a crew of astronauts who travel through a newly discovered wormhole in search of habitable planets to replace a dying Earth\n- Known for its scientifically accurate depiction of concepts like black holes, relativity, and theoretical physics\n- Acclaimed for its visuals, score, and emotional storytelling\n- Won the Academy Award for Best Visual Effects\n- Considered one of Christopher Nolan\'s best and most ambitious films\n\nOther highly regarded sci-fi movies include "2001: A Space Odyssey," "Blade Runner," "The Matrix," "Arrival," and "Inception." But "Interstellar" stands out for its grand scale, scientific authenticity, and Nolan\'s directorial vision.'

## Chain and LangChain Expressions Language (LCEL)

<div style="background-color:#f0f8ff; padding: 15px; border-radius: 10px; border-left: 6px solid #4682B4;">
  <p><b>LangChain Expression Language (LCEL)</b> is a declarative language for composing LangChain components. LangChain compiles LCEL compositions into an optimized execution plan, offering automatic parallelization, streaming, tracing, and async support.</p>
</div>


#### Simple Sequential Chain

In [17]:
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate

llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", model_kwargs={"temperature": .1})

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a travel guide. Welcome to the {city} travel guide!
        If you're visiting in {month}, here's what you can do:
        1. Must-visit attractions.
        2. Local cuisine you must try.
        3. Useful phrases in {language}.
        4. Tips for traveling on a {budget} budget.
        Enjoy your trip!"""),
        ("human", "{user_input}")
    ]
)

chain = prompt_template | llm

# Get user input
city = input("Enter the city you're visiting: ")
month = input("Enter the month of your visit: ")
language = input("Enter the local language: ")
budget = input("Enter your budget (low/medium/high): ")
user_input = input("Enter any specific questions or requests: ")

response = chain.invoke({
    "city": city,
    "month": month,
    "language": language,
    "budget": budget,
    "user_input": user_input
})

print(response.content)

Enter the city you're visiting:  NYC
Enter the month of your visit:  Dec
Enter the local language:  English
Enter your budget (low/medium/high):  medium
Enter any specific questions or requests:  Adventure


Here are some adventurous things to do in NYC in December:

1. Must-Visit Attractions:
- Take a walking tour across the Brooklyn Bridge for amazing skyline views
- Go ice skating at iconic rinks like Rockefeller Center or Bryant Park
- Visit the Top of the Rock observation deck for panoramic city views

2. Local Cuisine:
- Try a classic NYC pizza slice from places like Joe's Pizza or Prince Street Pizza
- Get bagels and lox (cured salmon) from a legendary bagel shop like Russ & Daughters
- Sample global street foods from vendors and food trucks

3. Useful English Phrases:
- "Can you give me directions to..."
- "How much is the admission/cover charge?"
- "Where is the nearest subway station?"

4. Budget Travel Tips:
- Use the affordable subway system to get around 
- Look for free days/times at top museums and attractions
- Eat at casual delis, diners, and food trucks for cheap eats
- Consider the NYC City Pass for bundled attraction discounts

With its endless energy and variety of exp

#### Sequential Chain

In [18]:
# Generate a sequential chain with multiple chain for a blog generator
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0")

title_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a professional blogger.
        Create an outline for a blog post on the following topic: {topic}
        The outline should include:
        - 3 main points with subpoints
        - Conclusion
        Answer with the outline only, no additional text."""),
        ("human", "{topic}")
    ]
)

introduction_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a professional blogger.
        Write an engaging introduction paragraph based on the following
        outline:{outline}
        The introduction should hook the reader and provide a brief
        overview of the topic Also add the title of the blog post"""),
        ("human", "{outline}")
    ]
)

first_chain = title_prompt | llm | StrOutputParser() 
second_chain = introduction_prompt | llm | StrOutputParser()
final_chain = (
                {"topic": lambda x: x, "outline": first_chain} 
                | introduction_prompt 
                | llm 
                | StrOutputParser()
            )

topic = input("Enter the topic: ")
response = final_chain.invoke(topic)
print(response)

Enter the topic:  Linear Regression


Title: Mastering Linear Regression: A Comprehensive Guide

Introduction:

In the realm of data analysis and machine learning, linear regression stands as a fundamental and widely-used technique for modeling the relationship between variables. Whether you're a data scientist, a researcher, or simply someone curious about the world of predictive modeling, understanding linear regression is essential. This blog post aims to provide a comprehensive guide to linear regression, exploring its definition, applications, underlying assumptions, and techniques for building, evaluating, and improving linear regression models.

Linear regression is a statistical method that establishes a linear relationship between a dependent variable (often referred to as the target or response variable) and one or more independent variables (also known as predictors or features). Its purpose is to identify the best-fitting line or hyperplane that describes the relationship between these variables, enabling predi

## Memory: Learn from Interactions

#### Adding messages to memory with `ConversationBufferMemory`

In [19]:
from langchain.memory import ConversationBufferMemory
from pprint import pprint

sample_memory = ConversationBufferMemory(memory_key="history")

In [20]:
sample_memory.load_memory_variables({})

{'history': ''}

In [21]:
sample_memory.chat_memory.add_user_message("What's the capital of France?")
sample_memory.chat_memory.add_ai_message("The capital of France is Paris.")

# Let's check the contents of the memory
pprint(sample_memory.load_memory_variables({}))

{'history': "Human: What's the capital of France?\n"
            'AI: The capital of France is Paris.'}


In [22]:
sample_memory.chat_memory.add_user_message("What's the capital of India?")
sample_memory.chat_memory.add_ai_message("The capital of India is New Delhi.")

# Let's check the contents of the memory
pprint(sample_memory.load_memory_variables({}))

{'history': "Human: What's the capital of France?\n"
            'AI: The capital of France is Paris.\n'
            "Human: What's the capital of India?\n"
            'AI: The capital of India is New Delhi.'}


#### `ConversationBufferMemory` with `LLMChain`

In [23]:
from langchain_aws import ChatBedrock
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["history", "human_input"],
    template="""You are a knowledgeable food expert having a chat with a human about world cuisines.
    Previous chat: {history}
    Human: {human_input}
    Food Expert: """
)

memory = ConversationBufferMemory(input_key="human_input", memory_key="history", return_messages=True)

llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0")

chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory, verbose=True)

# First question
response1 = chain.invoke({"human_input": "What's a popular dish in Italian cuisine?"})
print(response1['text'])




> Entering new LLMChain chain...
Prompt after formatting:
You are a knowledgeable food expert having a chat with a human about world cuisines.
    Previous chat: []
    Human: What's a popular dish in Italian cuisine?
    Food Expert: 


/var/folders/sg/qf1dw3cs4q5007gb2_9zd8600000gr/T/ipykernel_40678/1966144164.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory, verbose=True)



> Finished chain.
Here are some of the most popular and iconic dishes in Italian cuisine:

• Pizza Napoletana - The classic Neapolitan pizza with tomato sauce, mozzarella cheese, and fresh basil. Naples is considered the birthplace of pizza.

• Pasta Bolognese - Pasta with a rich, meaty ragù (meat-based sauce) made with beef, vegetables and tomatoes. It originated in Bologna.

• Lasagna alla Bolognese - Layers of pasta with Bolognese ragù, béchamel sauce and cheese, also from Bologna.

• Risotto alla Milanese - A creamy Arborio rice dish flavored with saffron, from Milan. Other popular risotto varieties include mushroom and seafood.

• Ossobuco alla Milanese - Cross-cut veal shanks braised with wine, vegetables and broth, from Lombardy. 

• Saltimbocca alla Romana - Thin veal cutlets with prosciutto and sage, a classic Roman dish.

• Tiramisù - The famous coffee-flavored dessert made with ladyfingers, mascarpone cheese and cocoa powder.

Italian cuisine showcases incredible regional d

In [24]:
# Second question
response2 = chain.invoke({"human_input": "What's the main ingredient in this dish?"})
print(response2['text'])



> Entering new LLMChain chain...
Prompt after formatting:
You are a knowledgeable food expert having a chat with a human about world cuisines.
    Previous chat: [HumanMessage(content="What's a popular dish in Italian cuisine?"), AIMessage(content='Here are some of the most popular and iconic dishes in Italian cuisine:\n\n• Pizza Napoletana - The classic Neapolitan pizza with tomato sauce, mozzarella cheese, and fresh basil. Naples is considered the birthplace of pizza.\n\n• Pasta Bolognese - Pasta with a rich, meaty ragù (meat-based sauce) made with beef, vegetables and tomatoes. It originated in Bologna.\n\n• Lasagna alla Bolognese - Layers of pasta with Bolognese ragù, béchamel sauce and cheese, also from Bologna.\n\n• Risotto alla Milanese - A creamy Arborio rice dish flavored with saffron, from Milan. Other popular risotto varieties include mushroom and seafood.\n\n• Ossobuco alla Milanese - Cross-cut veal shanks braised with wine, vegetables and broth, from Lombardy. \n\n• Salt

In [25]:
from langchain_aws import ChatBedrock
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder

# Create the prompt template using ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "You are a knowledgeable food expert having a chat with a human about world cuisines."
    ),
    MessagesPlaceholder(variable_name="my_history"),
    HumanMessagePromptTemplate.from_template("{human_input}"),
])

# Set up memory
memory = ConversationBufferMemory(return_messages=True, memory_key="my_history")

# Initialize the LLM
llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0")

# Create the chain
chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory, verbose=True)

# First question
response1 = chain.invoke({"human_input": "What's the most popular dish in Italian cuisine? Pick only one."})
print(response1['text'])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a knowledgeable food expert having a chat with a human about world cuisines.
Human: What's the most popular dish in Italian cuisine? Pick only one.

> Finished chain.
If I had to pick just one dish as the most popular in Italian cuisine, I would say pizza.

Pizza is an iconic Italian food that has achieved immense global popularity. Its origins can be traced back to Naples, where the modern pizza we know today was developed in the late 18th or early 19th century. The combination of a thin crust, tomato sauce, and fresh mozzarella cheese became the classic Neapolitan pizza that captured the world's imagination.

While there are now countless variations, the simple Margherita pizza (named after the Queen consort of Italy, Margherita of Savoy) with just tomato, mozzarella, and basil remains one of the most beloved versions. Pizza has become so ingrained in Italian culture and cuisine that it's hard to imagine the 

In [26]:
# Second question
response2 = chain.invoke({"human_input": "What's the main ingredient in this dish?"})
print(response2['text'])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a knowledgeable food expert having a chat with a human about world cuisines.
Human: What's the most popular dish in Italian cuisine? Pick only one.
AI: If I had to pick just one dish as the most popular in Italian cuisine, I would say pizza.

Pizza is an iconic Italian food that has achieved immense global popularity. Its origins can be traced back to Naples, where the modern pizza we know today was developed in the late 18th or early 19th century. The combination of a thin crust, tomato sauce, and fresh mozzarella cheese became the classic Neapolitan pizza that captured the world's imagination.

While there are now countless variations, the simple Margherita pizza (named after the Queen consort of Italy, Margherita of Savoy) with just tomato, mozzarella, and basil remains one of the most beloved versions. Pizza has become so ingrained in Italian culture and cuisine that it's hard to imagine the country's food 

#### `ConversationBufferMemory` With `ConversationChain`


In [27]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0")

chain = ConversationChain(llm=llm, memory=memory, verbose=True)

response = chain.invoke({"input": "What's the most popular dish in Italian cuisine?"})
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What's the most popular dish in Italian cuisine?
AI:


/var/folders/sg/qf1dw3cs4q5007gb2_9zd8600000gr/T/ipykernel_40678/997765492.py:8: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  chain = ConversationChain(llm=llm, memory=memory, verbose=True)



> Finished chain.
{'input': "What's the most popular dish in Italian cuisine?", 'history': '', 'response': "Hmm, that's a great question! Italian cuisine is so rich and varied that it's hard to pinpoint one single most popular dish. Some contenders that come to mind are:\n\nPizza - This iconic flatbread with tomato sauce and mozzarella cheese originated in Naples and has become one of the most beloved Italian foods around the world. There are endless varieties from the classic Margherita to creative gourmet toppings.\n\nPasta - Pasta dishes like spaghetti bolognese, carbonara, and lasagna are quintessential Italian comfort foods enjoyed across the country and internationally. Each region has its own specialties when it comes to pasta shapes and sauces.\n\nRisotto - This creamy rice dish cooked with broth originated in northern Italy. Popular varieties include risotto alla milanese with saffron and risotto ai funghi with mushrooms.\n\nTiramisu - For dessert, the layered coffee-flavored

In [28]:
#pprint(memory.load_memory_variables({}))

#### `ConversationBufferMemory` With `RunnableWithMessageHistory`


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_aws import ChatBedrock
from langchain_core.chat_history import BaseChatMessageHistory

# Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Initialize the LLM
llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0")

# Create the runnable
runnable = prompt | llm

store = {} 

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Wrap the chain with message history
chain_with_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

# Use the chain
response1 = chain_with_history.invoke(
    {"input": "What is photosynthesis?"},
    config={'configurable': {'session_id': 'session_1'}}
)
print(response1.content)

Photosynthesis is the process by which plants and some other organisms convert light energy from the sun into chemical energy in the form of glucose (sugar) and oxygen. It occurs in the chloroplasts of plant cells.

The overall equation for photosynthesis is:

6CO2 + 6H2O + Sunlight Energy → C6H12O6 + 6O2

Where:
- CO2 is carbon dioxide
- H2O is water
- C6H12O6 is glucose
- O2 is oxygen

The process involves two main stages:

1) The Light Reactions: During this stage, light energy is absorbed and used to split water molecules into electrons, hydrogen ions, and oxygen gas. This provides the energy carrier molecules ATP and NADPH.

2) The Calvin Cycle: This stage doesn't directly use sunlight. It takes the ATP, NADPH, and carbon dioxide from the atmosphere and uses enzymes to produce glucose molecules. This is the photosynthetic carbon fixation phase.

Oxygen released during photosynthesis comes from splitting water molecules during the light reactions. Plants and algae produce most of t

In [ ]:
response2 = chain_with_history.invoke(
    {"input": "What?"},
    config={'configurable': {'session_id': 'session_1'}}
)
print(response2.content)



Let me explain photosynthesis in simpler terms:

Photosynthesis is how plants make their own food (glucose/sugar) using sunlight, water and carbon dioxide from the air.

It happens in the green parts of plants, mainly the leaves, which contain a green pigment called chlorophyll.

During photosynthesis:

1) The plant absorbs light energy from the sun.

2) It takes in carbon dioxide from the air through tiny pores in its leaves. 

3) It also takes in water from the soil through its roots.

4) Using the energy from sunlight, the plant combines the carbon dioxide and water to produce glucose (food for the plant) and releases oxygen as a byproduct.

5) The oxygen is released out into the air through the leaf pores.

So in simple terms - plants use sunlight, carbon dioxide and water to produce their food (glucose) and release oxygen in the process, which humans and animals need to breathe.

Does this help explain what photosynthesis is in an easier to understand way? Let me know if you need 

In [ ]:
response3 = chain_with_history.invoke(
    {"input": "What still can't get it? Can you explain it in a simpler way?"},
    config={'configurable': {'session_id': 'session_1'}}
)
print(response3.content)

Sure, let me try to explain photosynthesis in the simplest way possible:

Photosynthesis is like a plant's way of cooking its own food.

Just like we need ingredients to cook food in the kitchen, plants need three main "ingredients" for photosynthesis:

1) Sunlight - This provides the energy/heat for "cooking."
2) Water - Plants take this up from the soil through their roots.
3) Carbon dioxide - Plants take this gas from the air through their leaves.

Using the energy from sunlight, plants mix the water and carbon dioxide together inside their leaves to make a sugar called glucose. Glucose is the "food" that gives plants energy to grow.

But when plants "cook" this glucose food, they also make a leftover "ingredient" that they don't need - oxygen gas. So plants release this oxygen back out into the air through tiny holes in their leaves.

This oxygen is very important for animals and humans to breathe.

So in simple terms - plants use sunlight, water and air (carbon dioxide) to make th

#### `ConversationBufferWindowMemory`

In [29]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

memory = ConversationBufferWindowMemory(k=1)

llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0")

convo_chain = ConversationChain(llm=llm, memory=memory)

convo_chain.invoke({"input": "Whats the capital of India?"})

{'input': 'Whats the capital of India?',
 'history': '',
 'response': "The capital of India is New Delhi.\n\nNew Delhi is a city located in northern India and serves as the capital of the country. It is one of the largest cities in India with a population of over 29 million in the greater metropolitan area.\n\nSome key facts about New Delhi:\n\n- It has been the official capital of India since 1912 when the British moved the capital from Calcutta (now Kolkata).\n\n- It is home to many important government buildings and monuments like the Indian Parliament, Rashtrapati Bhavan (the Presidential Palace), India Gate war memorial, and more.\n\n- It encompasses two distinct areas - New Delhi which is the planned city designed by British architects, and Old Delhi which is the ancient walled city dating back centuries.\n\n- Major attractions include the Red Fort, Jama Masjid mosque, Chandni Chowk market, Qutub Minar tower, Humayun's Tomb and others.\n\n- It has an international airport connect

In [30]:
convo_chain.invoke({"input": "Whats the capital of France?"})

{'input': 'Whats the capital of France?',
 'history': "Human: Whats the capital of India?\nAI: The capital of India is New Delhi.\n\nNew Delhi is a city located in northern India and serves as the capital of the country. It is one of the largest cities in India with a population of over 29 million in the greater metropolitan area.\n\nSome key facts about New Delhi:\n\n- It has been the official capital of India since 1912 when the British moved the capital from Calcutta (now Kolkata).\n\n- It is home to many important government buildings and monuments like the Indian Parliament, Rashtrapati Bhavan (the Presidential Palace), India Gate war memorial, and more.\n\n- It encompasses two distinct areas - New Delhi which is the planned city designed by British architects, and Old Delhi which is the ancient walled city dating back centuries.\n\n- Major attractions include the Red Fort, Jama Masjid mosque, Chandni Chowk market, Qutub Minar tower, Humayun's Tomb and others.\n\n- It has an inter

In [31]:
convo_chain.invoke({"input": "Whats the capital of Italy?"})

{'input': 'Whats the capital of Italy?',
 'history': 'Human: Whats the capital of France?\nAI: The capital of France is Paris.\n\nParis is one of the most famous and iconic cities in the world. Some key facts about Paris:\n\n- It has been the capital of France since the late 6th century during the reign of the Merovingian kings.\n\n- It is located in the Île-de-France region in the north-central part of the country, straddling the Seine river.\n\n- With a population of over 2 million in the city proper and nearly 12 million in the larger metropolitan area, it is one of the most populous cities in Europe.\n\n- Major landmarks include the Eiffel Tower, the Louvre museum, the Notre-Dame Cathedral, the Arc de Triomphe, the Champs-Élysées avenue and many more.\n\n- It is an global center for art, fashion, gastronomy, and culture. The Louvre is the most visited art museum in the world.\n\n- Paris hosts the annual French Open tennis tournament at Roland Garros stadium.\n\n- The city has a ver

In [32]:
#print(convo_chain.memory.buffer)

#### `ConversationSummaryMemory`

In [33]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationChain
from langchain_aws import ChatBedrock

llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0")

memory = ConversationSummaryMemory(llm=llm)

convo_chain = ConversationChain(llm=llm, memory=memory)

convo_chain.invoke({"input": "Whats the capital of India?"})
convo_chain.invoke({"input": "Whats the capital of France?"})
convo_chain.invoke({"input": "Whats the capital of Italy?"})

{'input': 'Whats the capital of Italy?',
 'history': "The human asks what the capital of India is, and the AI responds that it is New Delhi. The AI provides additional details about New Delhi, including that it is a major city located in northern India with a population over 29 million in the metropolitan area. The AI notes that New Delhi was established in 1911 by the British rulers as the new capital, replacing Calcutta (Kolkata). The city's architecture and urban planning was influenced by British architects and features grand government buildings and monuments. The AI mentions some key landmarks like India Gate, the Qutub Minar, Lotus Temple, and Red Fort. Delhi, encompassing Old Delhi and New Delhi, is considered one of the world's oldest continuously inhabited cities.\n\nThe human then asks what the capital of France is, and the AI states it is Paris. The AI provides details on Paris, noting it is a global center for culture, arts, fashion, gastronomy and more. It highlights famo